# MNIST Training using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [1]:
install_needed = True  # should only be True once
# install_needed = False

In [2]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [3]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install --upgrade pip 
    !{sys.executable} -m pip install -U sagemaker smdebug sagemaker-experiments
    # !{sys.executable} -m pip install -U torchvision
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached sagemaker-2.143.0.tar.gz (706 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.143.0-py2.py3-none-any.whl size=948774 sha256=510254b47bc8832a0942499cf389def0cfd0a9da95145b4be10a4fd8867512c4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/bb/8e/cd/6c0c23e65b4788ae22f03739caa450dc75492c45ec5ce6697d
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.140.0
    Uninstalling sagemaker-2.140.0:
      Successfully uninstalled sagemaker-2.140.0


In [106]:
import sagemaker

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial

from sagemaker.debugger import (
    Rule, ProfilerRule, rule_configs, ProfilerConfig, 
    FrameworkProfile, DetailedProfilingConfig, 
    DataloaderProfilingConfig, PythonProfilingConfig)
import time

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

In [107]:
sagemaker.__version__

'2.143.0'

## Data
### Getting the data



In [108]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [109]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-322537213286/sagemaker/DEMO-pytorch-mnist


## Train
### Training script
The script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

### Configure Experiment
Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker.

In [110]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

### Configure Trial

In [111]:
def create_trial(experiment_name):
    from time import strftime
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

## Setup fsx and use fsx for data channels and checkpoints

While the above option is easier to setup, using an FSX can be beneficial for performance when dealing with large input sizes and large model sizes. 

Please see the instructions [here](https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb), to create the FSx lustre filesystem and import the dataset from the S3 bucket to your fsx filesystem. Note that the FSX must be created in a private subnet with internet gateway to ensure that training job has access to the internet. 

In [112]:
# # # Instructions obtained from:

# use_fsx = False

# if use_fsx:
#     from sagemaker.inputs import FileSystemInput

#     # Specify FSx Lustre file system id.
#     # file_system_id = "<your-file-system-id>"
#     file_system_id = 'fs-099a02ed242945403'

#     # Specify directory path for input data on the file system.
#     # You need to provide normalized and absolute path below.
#     # Your mount name can be provided by you when creating fsx, or generated automatically.
#     # You can find this mount_name on the FSX page in console.
#     # Example of fsx generated mount_name: "3x5lhbmv"
#     # base_path = "<your-mount-name>"
#     base_path = '/q7rzxbmv'

#     # Specify your file system type.
#     file_system_type = "FSxLustre"

#     training = FileSystemInput(file_system_id=file_system_id,
#                             file_system_type=file_system_type,
#                             directory_path=base_path + "/git-dataset",
#                             file_system_access_mode="rw")
#     model = FileSystemInput(file_system_id=file_system_id,
#                             file_system_type=file_system_type,
#                             directory_path=base_path + "/bloom_model/bloom-560m",
#                             file_system_access_mode="rw")

#     data_channels = {"training": training, "model": model}

### Configure rules
We specify the following rules:
- loss_not_decreasing: checks if loss is decreasing and triggers if the loss has not decreased by a certain persentage in the last few iterations
- LowGPUUtilization: checks if GPU is under-utilizated 
- ProfilerReport: runs the entire set of performance rules and create a final output report with further insights and recommendations.

In [113]:
rules=[
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

### Create a Training Job with Profiling Enabled<a class="anchor" id="option-1"></a>

You will use the standard [SageMaker Estimator API for PyTorch ](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.pytorch.html) to create training jobs. To enable profiling, create a `ProfilerConfig` object and pass it to the `profiler_config` parameter of the `PyTorch` estimator.

In [114]:
profiler_config=ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(
        start_step=5,num_steps=10,
        detailed_profiling_config=DetailedProfilingConfig(start_step=5, num_steps=10),
        dataloader_profiling_config=DataloaderProfilingConfig(start_step=5, num_steps=10),
        python_profiling_config=PythonProfilingConfig(start_step=5, num_steps=10), # cprofile / Pyinstrument
    )
)

In [115]:
metric_definitions=[
     {'Name': 'train:Loss', 'Regex': 'Train Loss: (.*?),'},
     {'Name': 'test:Loss', 'Regex': 'Average loss: (.*?),'},
     {'Name': 'test:Accuracy', 'Regex': 'Accuracy: (.*?)%'},
]

### Model training with SMDataParallel


The training script provides the code you need for distributed data parallel (DDP) training using SMDataParallel. The training script is very similar to a PyTorch training script you might run outside of SageMaker, but modified to run with SMDataParallel. SMDataParallel's PyTorch client provides an alternative to PyTorch's native DDP. For details about how to use SMDataParallel's DDP in your native PyTorch script, see the Getting Started with SMDataParallel tutorials.

In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

SMDataParallel supports model training on SageMaker with the following instance types only:
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of SMDataParallel, you should use at least 2 instances, but you can also use 1 for testing this example.

In [123]:
experiment_name = 'mnist-exp1'
instance_type = 'ml.p3.16xlarge' # 'ml.p3.2xlarge'  # 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge', 'local_gpu'
instance_type = 'local_gpu'
instance_count = 1
use_spot_instances = False
max_wait = None
max_run = 1*60*60

In [124]:
if instance_type =='local_gpu':
    from sagemaker.local import LocalSession
    from pathlib import Path

    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    s3_data_path = f'file://{Path.cwd()}/data'
    source_dir = f'{Path.cwd()}/train_code'
else:
    sagemaker_session = sagemaker.Session()
    s3_data_path = inputs
    source_dir = 'train_code'


**Distribution strategy**

Note that to use DDP mode, you update the the `distribution` strategy, and set it to use `smdistributed dataparallel`. 

In [125]:
# distribution = {}

# ### MPIRUN 수행
# distribution["mpi"]={"enabled": True}


### SageMaker DDP
# distribution["smdistributed"] = {
#     "dataparallel": {"enabled": True}
# }

# ### SageMaker MP
# distribution['smdistributed'] = {
#     "modelparallel": {"enabled":True,
#                       "parameters": {
#                           "partitions": mp_parameters['num_partitions'],
#                           "microbatches": mp_parameters['num_microbatches'],
#                           "placement_strategy": mp_parameters['placement_strategy'],
#                           "pipeline": mp_parameters['pipeline'],
#                           "optimize": mp_parameters['optimize'],
#                           "memory_weight": mp_parameters['memory_weight'],
#                           "ddp": mp_parameters['ddp']
#                       }
#                      }
# }

In [126]:
distribution = {}

flag = None
# flag = 'ddp'
# flag = 'smddp'

if flag == 'ddp':
    distribution["mpi"]={"enabled": True}
    # entry_point='pytorch_mnist_ddp.py'
elif flag == 'smddp':
    distribution["smdistributed"] = {"dataparallel": {"enabled": True}}
    # entry_point='pytorch_mnist_smddp.py'
else:
    distribution = None
    # entry_point='pytorch_mnist_sm.py'
distribution

{'smdistributed': {'dataparallel': {'enabled': True}}}

In [127]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='pytorch_mnist.py',
                    source_dir=source_dir,
                    role=role,
                    framework_version='1.13.1',
                    py_version='py39',
                    instance_count=instance_count,
                    instance_type=instance_type,
                    distribution=distribution,
                    metric_definitions=metric_definitions,
                    profiler_config=profiler_config,
                    rules=rules,
                    # disable_profiler=True,
                    use_spot_instances=use_spot_instances,
                    max_wait=max_wait,
                    max_run=max_run,
                    hyperparameters={
                        'epochs': 5,
                                    },
                    sagemaker_session=sagemaker_session,
                    tags=[{'Key':'Owner','Value':'username'}]
                   )

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [128]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name)


estimator.fit(
    inputs={'training': s3_data_path}, 
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=False,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: mnist-exp1-0401-07381680334715


In [129]:
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

2023-04-01 07:38:38 Starting - Starting the training job...ProfilerReport: InProgress
...
2023-04-01 07:39:33 Starting - Preparing the instances for training......
2023-04-01 07:40:33 Downloading - Downloading input data......
2023-04-01 07:41:33 Training - Downloading the training image..................
2023-04-01 07:44:44 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-04-01 07:45:09,899 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-04-01 07:45:09,965 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-01 07:45:09,978 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-04-01 07:45:09,980 sagemaker_pytorch_container.training INFO     Invoking SMDataParallel
2023-04-01 07:45:09,980 sagemaker_pytorch_container.trai

### Download Debugger Profiling Report
The profiling report rule will create an html report `profiler-report.html` with a summary of builtin rules and recommenades of next steps. You can find this report in your S3 bucket. For more information about how to download and open the Debugger profiling report, see [SageMaker Debugger Profiling Report](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-profiling-report.html) in the SageMaker developer guide.

In [130]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

You will find the profiler report in s3://sagemaker-us-west-2-322537213286/mnist-exp1-0401-07381680334715/rule-output


In [131]:
!aws s3 ls {rule_output_path}/ProfilerReport/profiler-output/

                           PRE profiler-reports/
2023-04-01 07:48:11     351823 profiler-report.html
2023-04-01 07:48:11     196930 profiler-report.ipynb


In [132]:
import os

output_dir = './output'
!rm -rf $output_dir

profile_output = output_dir+'/ProfilerReport'

if not os.path.exists(profile_output):
    os.makedirs(profile_output)

In [133]:
!aws s3 cp {rule_output_path}/ProfilerReport/profiler-output/ {output_dir}/ProfilerReport/ --recursive

download: s3://sagemaker-us-west-2-322537213286/mnist-exp1-0401-07381680334715/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json to output/ProfilerReport/profiler-reports/CPUBottleneck.json
download: s3://sagemaker-us-west-2-322537213286/mnist-exp1-0401-07381680334715/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json to output/ProfilerReport/profiler-reports/BatchSize.json
download: s3://sagemaker-us-west-2-322537213286/mnist-exp1-0401-07381680334715/rule-output/ProfilerReport/profiler-output/profiler-report.html to output/ProfilerReport/profiler-report.html
download: s3://sagemaker-us-west-2-322537213286/mnist-exp1-0401-07381680334715/rule-output/ProfilerReport/profiler-output/profiler-reports/LowGPUUtilization.json to output/ProfilerReport/profiler-reports/LowGPUUtilization.json
download: s3://sagemaker-us-west-2-322537213286/mnist-exp1-0401-07381680334715/rule-output/ProfilerReport/profiler-output/profiler-reports/IOBottleneck.js

In [134]:
from IPython.core.display import display, HTML

display(HTML('<b>ProfilerReport : <a href="{}profiler-report.html">Profiler Report</a></b>'.format(output_dir+"/ProfilerReport/")))
